#### 1. Bibliotecas

In [ ]:
# Importacao de bibliotecas

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

#### 2. Importacao de dados

leitura em excel .xlsx, mas também pode ser .csv

In [ ]:
# dados de saida

dados_saida = pd.read_excel('path_do_arquivo_aqui')

# dados de entrada

dados_entrada = pd.read_excel('path_do_arquivo_aqui')

Informações das tabelas

In [ ]:
dados_saida.info()

In [ ]:
dados_entrada.info()

#### 3. Manipulação inicial dos dados

para as datas entre 01/01/2023 à 31/12/2023


In [ ]:
# Vetor com todas as datas de 2023

datas_2023 = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

Valores de saídas diários

In [ ]:
dados_saida_valor_agrupados = dados_saida.groupby('Data', as_index=False)['Valor_total'].sum()

df_saida_data_valor_total = pd.DataFrame({'Data': datas_2023})

df_saida_data_valor_total = pd.merge(df_saida_data_valor_total, dados_saida_valor_agrupados, how='left', on='Data')

df_saida_data_valor_total['Valor_total'].fillna(0, inplace=True)

In [ ]:
df_saida_data_valor_total

Valores de entradas diários

In [ ]:
dados_entrada_valor_agrupados = dados_entrada.groupby('Data_pagamento', as_index=False)['Valor'].sum()

dados_entrada_valor_agrupados = dados_entrada_valor_agrupados.rename(columns={'Data_pagamento': 'Data'})
dados_entrada_valor_agrupados = dados_entrada_valor_agrupados.rename(columns={'Valor': 'Valor_total'})

df_entrada_data_valor_total = pd.DataFrame({'Data': datas_2023})

df_entrada_data_valor_total = pd.merge(df_entrada_data_valor_total, dados_entrada_valor_agrupados, how='left', on='Data')

df_entrada_data_valor_total['Valor_total'].fillna(0, inplace=True)

In [ ]:
df_entrada_data_valor_total

Valores de entradas e saídas diários e acumulados

In [ ]:
df_entrada_saidas_data_valor = pd.DataFrame({'Data': datas_2023})

df_entrada_saidas_data_valor = pd.merge(df_entrada_saidas_data_valor, df_entrada_data_valor_total, how='left', on='Data')

df_entrada_saidas_data_valor = pd.merge(df_entrada_saidas_data_valor, df_saida_data_valor_total, how='left', on='Data')

df_entrada_saidas_data_valor = df_entrada_saidas_data_valor.rename(columns={'Valor_total_x': 'Valor_total_entrada'})

df_entrada_saidas_data_valor = df_entrada_saidas_data_valor.rename(columns={'Valor_total_y': 'Valor_total_saida'})

df_entrada_saidas_data_valor['Valor_entrada_acumulado'] = df_entrada_saidas_data_valor['Valor_total_entrada'].cumsum()

df_entrada_saidas_data_valor['Valor_saida_acumulado'] = df_entrada_saidas_data_valor['Valor_total_saida'].cumsum()

In [ ]:
df_entrada_saidas_data_valor

Balanço mensal de saldo

In [ ]:
df_entrada_saidas_mensal = dados_saida.groupby(dados_saida['Data'].dt.to_period('M'))['Valor_total'].sum()

df_saida_saidas_mensal = dados_entrada.groupby(dados_entrada['Data_pagamento'].dt.to_period('M'))['Valor'].sum()

balanco_mensal_total = df_saida_saidas_mensal - df_entrada_saidas_mensal

#### 4. Gráficos de Análise Macro


Gráfico de Linhas dos Gastos Diários

In [ ]:
plt.figure(figsize=(25, 5))

plt.plot(df_saida_data_valor_total['Data'], 
         df_saida_data_valor_total['Valor_total'], 
         linewidth=2, 
         linestyle='-', 
         color='red')

plt.title('GASTOS POR DIA')
plt.xlabel('Período')
plt.ylabel('Valores em R$')

plt.grid(False)

plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(True)
plt.gca().spines['bottom'].set_visible(True)

plt.show()

plt.savefig(r'local_para_salvar_arquivo_aqui.png', dpi=500, bbox_inches='tight')

Gráfico de Linhas dos Entradas Diários

In [ ]:
plt.figure(figsize=(25, 5))

plt.plot(df_entrada_data_valor_total['Data'], 
         df_entrada_data_valor_total['Valor_total'], 
         linewidth=2, 
         linestyle='-', 
         color='#B3B3B3')

plt.title('ENTRADAS POR DIA')
plt.xlabel('Período')
plt.ylabel('Valores (R$)')

plt.grid(False)

plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(True)
plt.gca().spines['bottom'].set_visible(True)

plt.show()

plt.savefig(r'local_para_salvar_arquivo_aqui.png', dpi=500, bbox_inches='tight')

Caso deseje fazer uma divisão dos valores por período para, por exemplo, plotar com cores e legendas diferentes, aplicar a seguinte logica

In [ ]:
periodo1 = df[(df['Data'] >= pd.to_datetime('2023-01-01')) & (df['Data'] <= pd.to_datetime('2023-01-31'))]
periodo2 = df[(df['Data'] >= pd.to_datetime('2023-01-31')) & (df['Data'] <= pd.to_datetime('2023-02-28'))]

plt.plot(periodo1['Data'], periodo1['Valor_total'], linestyle='--', linewidth=2, color='#B3B3B3', label='LEGENDA PERIODO 1')
plt.plot(periodo2['Data'], periodo2['Valor_total'], linestyle='-', linewidth=2, color='#FD7524', label='LEGENDA PERIODO 2')


Gráfico de Linhas dos Entradas e Saídas Acumuladas

In [ ]:
plt.figure(figsize=(25, 10))

plt.plot(df_entrada_saidas_data_valor['Data'], 
         df_entrada_saidas_data_valor['Valor_entrada_acumulado'],
         linestyle='--',
         linewidth=2,
         color='green', 
         label='Entradas')

plt.plot(df_entrada_saidas_data_valor['Data'], 
         df_entrada_saidas_data_valor['Valor_saida_acumulado'],
         linestyle='-', 
         linewidth=2,
         color='red', 
         label='Saidas')

plt.title('ENTRADAS E SAÍDAS ACUMULADAS')
plt.xlabel('Período')
plt.ylabel('Valores (R$)')
plt.grid(False)

plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(True)
plt.gca().spines['bottom'].set_visible(True)

# Marca de valores de saida
marca_valores = [5000, 10000, 15000, 20000]

# Datas dos valores marcados
marca_valores_datas = [df_entrada_saidas_data_valor.loc[(df_entrada_saidas_data_valor['Valor_saida_acumulado'] >= value), 
                                                 'Data'].min() for value in marca_valores]

for date, value in zip(marca_valores_datas, marca_valores):
    plt.plot(date, value, marker='o', color='red')

print(marca_valores_datas)

plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=10, title="")

plt.show()

plt.savefig(r'local_para_salvar_arquivo_aqui.png', dpi=500, bbox_inches='tight')

Gráfico de Barras de Saldo = Entradas - Saidas

In [ ]:
plt.figure(figsize=(25, 10))

ax = balanco_mensal_total.plot(kind='bar', color=colors)

ax.set_xticklabels([month.strftime('%b') for month in balanco_mensal_total.index], rotation=0)

colors = ['#13A555' if balance >= 0 else '#FF0000' for balance in balanco_mensal_total]

for i, balance in enumerate(balanco_mensal_total):
    ax.text(i, balance, f'R$ {balance:.2f}', ha='center', va='bottom')

plt.title('BALANÇO MENSAL')
plt.xlabel('Período')
plt.ylabel('Valores (R$)')

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(True)
plt.gca().spines['bottom'].set_visible(True)

plt.show()

plt.savefig(r'local_para_salvar_arquivo_aqui.png', dpi=500, bbox_inches='tight')

#### 5. Gráficos de Análise Micro

Dados de origem das ENTRADAS

In [ ]:
# Manipulação dos dados

dados_origem_valor = dados_entrada.groupby('Origem', as_index=False)['Valor'].sum()

dados_origem_valor = dados_origem_valor.sort_values(by='Valor', ascending=False)

# Top 5 origens de entradas e demais
top5_origem_valor = pd.concat([dados_origem_valor.head(5),
                                  pd.DataFrame({'Origem': ['Outros'], 'Valor': [dados_origem_valor['Valor'][5:].sum()]})],
                                  ignore_index=True)

In [ ]:
# Gráfico de Pizza

plt.figure(figsize=(8, 8))

wedges, texts, autotexts = plt.pie(top5_origem_valor['Valor'], 
                                   autopct='%1.2f%%', 
                                   startangle=0, 
                                   pctdistance=1.15, 
                                   colors = ['#004EA7', '#FF3535', '#36CD00', '#FD7524', '#8100FF', '#B3B3B3'],
                                   wedgeprops=dict(width=0.4, edgecolor='white'))

plt.setp(autotexts, size=12, weight="bold")

legendas_origens = [f"{origem} | R$ {valor:.2f}" for origem, valor in zip(top5_origem_valor['Origem'], top5_origem_valor['Valor'])]

plt.legend(legendas_origens, loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12, title="ORIGENS")

ax = plt.gca()
ax.annotate(f"TOTAL\nR$ {top5_origem_valor['Valor'].sum():.2f}", 
            xy=(0.5, 0.5), 
            xycoords='axes fraction', 
            ha='center', 
            va='center', 
            fontsize=12,
            weight="bold",
            bbox=dict(boxstyle="square", alpha=0))

plt.title('DISTRIBUIÇÃO DAS ORIGENS DOS VALORES DE ENTRADA')

plt.show()

plt.savefig(r'local_para_salvar_arquivo_aqui.png', dpi=500, bbox_inches='tight')

Dados de categoria das SAÍDAS

In [ ]:
# Manipulação dos dados

dados_categoria_valor = dados_saida.groupby('Categoria', as_index=False)['Valor_total'].sum()

dados_categoria_valor = dados_categoria_valor.sort_values(by='Valor_total', ascending=False)

dados_categoria_valor = dados_categoria_valor.rename(columns={'Valor_total': 'Valor'})

# Top 5 categorias de saidas e demais
top5_categoria_valor = pd.concat([dados_categoria_valor.head(5),
                                  pd.DataFrame({'Categoria': ['Outros'], 'Valor': [dados_categoria_valor['Valor'][10:].sum()]})],
                                  ignore_index=True)

In [ ]:
# Gráfico de Pizza

plt.figure(figsize=(8, 8))

wedges, texts, autotexts = plt.pie(top5_categoria_valor['Valor'], 
                                   autopct='%1.2f%%', 
                                   startangle=0, 
                                   pctdistance=1.15, 
                                   colors=['#5ADA58', '#DA589F', '#587DDA', '#8258DA', '#DA5858'],
                                   wedgeprops=dict(width=0.4, edgecolor='white'))


plt.setp(autotexts, size=12, weight="bold")

legendas_categorias = [f"{categoria} | R$ {valor:.2f}" for categoria, valor in zip(top5_categoria_valor['Categoria'], top5_categoria_valor['Valor'])]

plt.legend(legendas_categorias, loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12, title="CATEGORIAS")

ax = plt.gca()
ax.annotate(f"TOTAL\nR$ {top5_categoria_valor['Valor'].sum():.2f}", 
            xy=(0.5, 0.5), 
            xycoords='axes fraction', 
            ha='center', 
            va='center', 
            fontsize=12,
            weight="bold",
            bbox=dict(boxstyle="square", alpha=0))

plt.title('DISTRIBUIÇÃO DE CATEGORIAS DOS VALORES DE SAÍDA')

plt.show()

plt.savefig(r'local_para_salvar_arquivo_aqui.png', dpi=500, bbox_inches='tight')

Dados de Itens por Categorias das SAÍDAS

In [ ]:
def plot_top_itens_categoria(dados, categoria, n_top, local_save):

    # Manipulação dos dados

    df_iten_categoria = dados[dados['Categoria'] == categoria]
    
    df_categoria_valor = df_iten_categoria.groupby('Item', as_index=False)['Valor_total'].sum().sort_values(by='Valor_total', ascending=False) # type: ignore
    
    top_n_iten_categoria_valor = df_categoria_valor.head(n_top)

    display(top_n_iten_categoria_valor)

    # Gráfico de Pizza

    plt.figure(figsize=(8, 8))

    colors = ['#000000', '#1A1A1A', '#333333', '#4D4DA4D', '#666666', '#808080', '#999999', '#B3B3B3', '#CCCCCC', '#E6E6E6']

    wedges, texts, autotexts = plt.pie(top_n_iten_categoria_valor['Valor_total'], 
                                    autopct='%1.2f%%', 
                                    startangle=0, 
                                    pctdistance=1.15, 
                                    colors=colors, 
                                    wedgeprops=dict(width=0.4, edgecolor='white'))

    plt.setp(autotexts, size=12, weight="bold")

    legendas_itens = [f"{item} | R$ {valor:.2f}" for item, valor in zip(top_n_iten_categoria_valor['Item'], top_n_iten_categoria_valor['Valor_total'])]
    
    plt.legend(legendas_itens, loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=12, title="ITENS")

    ax = plt.gca()
    ax.annotate(f"TOTAL\nR$ {top_n_iten_categoria_valor['Valor_total'].sum():.2f}", 
                xy=(0.5, 0.5), 
                xycoords='axes fraction', 
                ha='center', 
                va='center', 
                fontsize=12,
                weight="bold",
                bbox=dict(boxstyle="square", alpha=0))

    plt.title(f"DISTRIBUIÇÃO DOS {n_top} ITENS DE {categoria}", fontsize=16, x=0.75, weight="bold")

    plt.show()

    plt.savefig(local_save, dpi=500, bbox_inches='tight')

Distribuição mensal dos valores por Categoria de SAÍDA (em porcentagem)

In [ ]:
def plot_dist_categoria_mensal(dados, categoria, cor, local_save):

    # Manipulação dos dados

    df_categoria_mensal = dados.groupby([dados['Data'].dt.to_period("M"), 'Categoria'])['Valor_total'].sum().reset_index()
    df_categoria_mensal['Data'] = df_categoria_mensal['Data'].dt.strftime('%b')

    df_valor_mensal_categoria = df_categoria_mensal.pivot(index='Data', columns='Categoria', values='Valor_total').fillna(0)

    ordem_meses = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    df_valor_mensal_categoria = df_valor_mensal_categoria.reindex(ordem_meses)

    ordem_alfabetica_categoria = sorted(df_valor_mensal_categoria.columns)

    paleta_cores_cinza = sns.color_palette("gray", n_colors=len(ordem_alfabetica_categoria))
    colors = {category: paleta_cores_cinza[i] for i, category in enumerate(ordem_alfabetica_categoria)}
    colors[categoria] = cor

    df_porcentagem = df_valor_mensal_categoria.divide(df_valor_mensal_categoria.sum(axis=1), axis=0) * 100

    df_porcentagem.plot(kind='bar', stacked=True, color=[colors[col] for col in df_porcentagem.columns],  width=0.75)

    plt.title('DISTRIBUIÇÃO MENSAL DAS CATEGORIAS DE SAÍDA', fontsize=16)

    plt.xlabel('Período')
    plt.ylabel('Porcentagem do Valor Total (%)')

    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['left'].set_visible(True)
    plt.gca().spines['bottom'].set_visible(True)

    plt.legend('', frameon=False)

    plt.savefig(local_save, dpi=500, bbox_inches='tight')